In [1]:
import sys
import numpy as np
import re

In [2]:
lang_ops = set({'FORK', 'JOIN', 'LOCK', 'READ', 'THREADEXIT', 'THREADINIT', 'UNLOCK', 'WRITE', 'NOTIFY','WAIT'})

In [3]:
regex_1= r"^( *)(\d+)( *):( *)([A-Z]+)( *)[\(]( *)([a-zA-Z0-9]+)( *),( *)([a-zA-Z0-9]+)( *)[\)]( *)$"
regex_2 = r"^( *)(\d+)( *):( *)([A-Z]+)( *)[\(]( *)([a-zA-Z0-9]+)( *)[\)]( *)$"

In [6]:
def check_semantics(trace):
    if trace[1] == 'FORK' and trace[2] in running and trace[3] not in created and trace[3] not in running and trace[3] not in finished:
        created.add(trace[3])
    elif trace[1] == 'JOIN' and trace[2] in running and trace[3] in finished:
        ;
    elif trace[1] == 'LOCK' and ( not locks.has_key(trace[3]) or locks.get(trace[3]) == ''):
            locks.update({trace[3]: trace[2]})
    elif trace[1] == 'READ' and trace[2] in running:
        ;
    elif trace[1] == 'THREADEXIT' and trace[2] in running:
        running.remove(trace[2])
        finished.add(trace[2])
    elif trace[1] == 'THREADINIT' and trace[2] in created:
        created.remove(trace[2])
        running.add(trace[2])
    elif trace[1] == 'UNLOCK' and locks.has_key(trace[3]) and locks.get(trace[3]) == trace[2]:
        locks.update({trace[3]:""})
    elif trace[1] == 'WRITE' and trace[2] in running:
        ;
    elif trace[1] == 'WAIT' and trace[2] in running:
        ;
    elif trace[1] == 'NOTIFY' and trace[2] in running:
        ;
    else:
        print "malformed trace @ " + str(trace)
        sys.exit(1)

def check_finish_semantics():
    if len(created) > 0 or len(running) > 0:
        print "malformed trace @ " + "incomplete trace"
        #sys.exit(1)

In [7]:
def graph_add_edge(source, destination):
    edges.add((source,destination))

In [39]:
def parser(literal):
    global global_index
    global_index+=1
    if re.match(regex_1,literal) is None and re.match(regex_2, literal) is None:
        print "malformed trace @" +literal
        sys.exit(1)
        
    instr_no=int(literal[:literal.find(":")].strip())
    if instr_no!=global_index:
        print "malformed trace @" +literal
        sys.exit(1)
    t=[]
    t.append(   instr_no )
    t.append( literal[ literal.find(":")+1 : literal.find("(") ].strip() )
    try:
        if t[1] not in lang_ops:
            raise Exception
            
        args=literal[literal.find("(")+1:literal.find(")")].strip().split(",")
        args=[x.strip() for x in args]
        if t[1] in ["THREADINIT", "THREADEXIT","NOTIFY"]:
            if len(args)!=1:
                raise Exception
        else:
            if len(args)!=2:
                raise Exception
        [t.append(x) for x in args]
        
    except :
        print "malformed trace @" +literal
        sys.exit(1)
    
    check_semantics(t)

    if thread_dict.has_key(t[2]):
        graph_add_edge(thread_dict[t[2]], t[0])
        thread_dict[t[2]] = t[0]
    else:
        thread_dict.update({t[2]: t[0]})
    try:
        
        if t[1] in  ["FORK", "UNLOCK"]:
            happens_before_dictionary[t[1]].update({t[3]:t[0]})
        elif t[1] in ["THREADEXIT","NOTIFY"]:
            happens_before_dictionary[t[1]].update({t[2]:t[0]})
        elif t[1] =="THREADINIT":
            graph_add_edge(happens_before_dictionary["FORK"].pop(t[2]),t[0])
        elif t[1] == "JOIN":
            graph_add_edge(happens_before_dictionary["THREADEXIT"].pop(t[3]),t[0])
        elif t[1] == "LOCK":
            graph_add_edge(happens_before_dictionary["UNLOCK"].pop(t[3]), t[0])    
    except KeyError:
        ;
    try:
        if t[1] == "WAIT":
            graph_add_edge(happens_before_dictionary["NOTIFY"][t[3]], t[0])
    except KeyError:
        print "malformed trace@ " + literal
        sys.exit(1)
        
    if t[1] in [ "READ", "WRITE"]:
        if objects.has_key(t[3]):
            temp = objects[t[3]]
            
            if type(temp) is list  and t[1]=="READ":
                objects[t[3]].append((t[2],1,t[0]))
                return
            if type(temp) is list and t[1] =="WRITE":
                for obj in temp:
                    possible_race.append((obj[2], t[0]))
                objects[t[3]] = (t[2],0,t[0])
            elif temp[1] ==0 and t[1] =="WRITE":
                possible_race.append((temp[2], t[0]))
                objects[t[3]] = (t[2],0,t[0])
            else:
                possible_race.append((temp[2], t[0]))
                objects[t[3]] = [(t[2],1,t[0])]
                
        else:
            if t[1] == "READ":
                objects.update({t[3] : [(t[2], 1, t[0])]})
            else:
                objects.update({t[3] : (t[2], 0, t[0])})

In [51]:
def trans_closure(edges):
    while True:
        relations = set([(a,b) for a,x in e for y,b in e if x==y])
        new_relations = relations | e
        if new_relations == edges:
            break
        edges = new_relations
    return edg

In [56]:
global_index=0
created=set({"t1"})
running=set()
finished=set()
locks = dict()
thread_locks = dict()
objects = dict() #{object_name : (thread_last_accessed, read:1/write:0, instruction_no)}

thread_dict = dict()
happens_before_dictionary = dict({"FORK" : dict(), "THREADEXIT" : dict(),"UNLOCK" : dict(), "NOTIFY" : dict() } )

possible_race=[]
edges = set()

In [57]:
f= open("./traces/TestCases/1_long.txt")
for row in f.readlines():
    if row.strip() in ['', None]:
        continue
    parser(row.strip())
f.close()

#check_finish_semantics()
del created,running,finished,locks,thread_locks,thread_dict,objects,happens_before_dictionary

possible_race = trans_closure(possible_race)
edges = trans_closure(edges)
set.difference(possible_race,edges)

KeyboardInterrupt: 

In [61]:
edges = trans_closure(edges)

set

In [10]:
possible_race = trans_closure(possible_race)

In [11]:
edges = trans_closure(edges)

In [76]:
len(set.difference(possible_race,edges))

18

In [10]:
objects

{'lck1': ('t1', 0, 18), 'obj1': ('t3', 1, 9), 'obj2': ('t1', 1, 21)}

In [16]:
thread_dict

{'t1': 22, 't2': 19, 't3': 16}

In [26]:
type([]) is list

True

In [78]:
while True:
    relations = set([(a,b) for a,x in edges for y,b in edges if x==y])
    new_relations = set.union(relations , edges)
    if new_relations == edges:
        break
    edges = new_relations

KeyboardInterrupt: 

In [93]:

d= dict() 
for k in range(852):
    for i in range(852):
        if k<i:
            d.update({(k+1,i+1):0})
    
for x,y in edges:
    d.update({(x,y):1})

In [89]:
del d

In [75]:
len(edges)

55083

In [94]:
for k in range(852):
    for i in range(852):
        for j in range(852):
            try:
                if d[(i+1,k+1)] ==1 and d[(k+1,j+1)] ==1:
                    d[(i+1,j+1)]=1
            except:
                ;
    print k

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [95]:


for a,b in possible_race:
    if d[(a,b)] !=1:
        print (a,b)

(4, 7)
(4, 113)
(476, 478)
(478, 849)
(112, 114)
